In [1]:
import math
import json
from json import JSONEncoder

class depotEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__

depot_name = "Tan Binh e-depot"
outfile = "data/etd.json"

In [2]:

class Point:
    def __init__(self,x,y) -> None:
        self.x = x
        self.y = y
    
    def translate(self, dx=0, dy=0):
        self.x += dx
        self.y += dy

    def scale(self, cox=1, coy=1):
        self.x *= cox
        self.y *= coy

class Shape:
    def __init__(self, seq) -> None:
        self.seq = seq
        self.length = len(seq)
    
    def scale(self,cox=1, coy=1):
        for i in range(self.length):
            self.seq[i].scale(cox,coy)
    
    def translate(self,dx=0, dy=0):
        for i in range(self.length):
            self.seq[i].translate(dx,dy)

class Container: 
    def __init__(self, contID, contSize, HangTau, Block, Bay, Row, Tier):
        self.ContID = contID
        self.ContTypeSizeID = contSize
        self.HangTauID = HangTau
        self.Block = Block
        self.Bay = Bay
        self.Row = Row
        self.Tier = Tier
        self.angle = 0
        self.x=0
        self.y=0
        self.z=0

In [3]:
def svg2layout(path):
    file = open(path, "r")
    str = file.read()
    layout_array = []
    a = 0
    p = str.find("<polygon",a)
    while p>0:
        x = []
        y = []
        b = str.find("/>",p)
        div = str[p+27:b-1] 
        print("shape:", div)
        chr_id=0
        type = 0
        sp = div.find(" ",chr_id)
        while sp>0:
            if type:
                # print("y:",div[chr_id:sp])
                y.append(int(div[chr_id:sp]))
            else:
                # print("x:",div[chr_id:sp])
                x.append(int(div[chr_id:sp]))

            type = 1-type
            chr_id = sp
            sp = div.find(" ", sp+1)
        # print("y:",div[chr_id:len(div)])
        y.append(int(div[chr_id:len(div)]))

        a = b
        p = str.find("<polygon",a)
        # print(x)
        # print(y)
        seq = []
        for i in range(len(x)):
            seq.append(Point(x[i],y[i]))
        layout_array.append(Shape(seq))
    return layout_array

In [4]:
class Area:
    def __init__(self, name, x, y, offsetX=0, offsetY=0, angle=0, x_flip=False, y_flip=False, shape=None, num_of_bay=15, num_of_row=6, num_of_tier=6) -> None:
        self.name = name
        self.num_of_bay = num_of_bay
        self.num_of_row = num_of_row
        self.num_of_tire = num_of_tier
        self.x_coor = x
        self.y_coor = y
        self.shape = shape
        self.angle = angle
        self.offset = Point(offsetX, offsetY)
        self.x_flip = x_flip
        self.y_flip = y_flip


class Layout:
    def __init__(self,name,svg_path) -> None:
        self.name = name
        self.shape = svg2layout(svg_path)

    def scale(self,cox=1, coy=1):
        for i in range(len(self.shape)):
            self.shape[i].scale(cox,coy)
    
    def translate(self,dx=0, dy=0):
        for i in range(len(self.shape)):
            self.shape[i].translate(dx,dy)

class House:
    def __init__(self, type, name, shape, id1=0, id2=2, height=100, angle=0) -> None:
        self.type = type
        self.name = name
        self.shape = shape
        self.id1 = id1
        self.id2 = id2
        self.height = height
        self.angle = angle


class Depot:
    def __init__(self, name, layout, area=[], house=[], container_scale=1) -> None:
        self.name = name
        self.layout = layout
        self.contWidth = 8 * container_scale
        self.contHeight = 8.5 * container_scale
        self.contLength = 20 * container_scale
        self.contHalfLength = 10 * container_scale
        self.contGap = math.floor(self.contLength*0.06)
        self.fontSize = 3*container_scale   
        self.Area = area
        self.house = house

In [5]:
1.48*2*math.pi/360


0.02583087292951608

In [6]:
etd_layout = Layout("General","data/tbd_ground.svg")
etd_layout.scale(cox=3, coy=3)
etd_layout.translate(-800, -1500)
# etd_layout.translate(2)

# add area here
Area_A = Area("A", etd_layout.shape[1].seq[6].x, etd_layout.shape[1].seq[6].y, x_flip=True)
Area_B = Area("B", etd_layout.shape[2].seq[4].x, etd_layout.shape[2].seq[4].y, x_flip=True)
Area_C = Area("C", etd_layout.shape[3].seq[0].x, etd_layout.shape[3].seq[0].y, angle=0.02583087292951608, offsetX=-42, offsetY=16)
Area_E = Area("D", etd_layout.shape[5].seq[1].x, etd_layout.shape[5].seq[1].y, angle=0.05235987755982988, offsetX=52, offsetY=5)
Area_SC = Area("SC", etd_layout.shape[4].seq[1].x, etd_layout.shape[4].seq[1].y, angle=0.05235987755982988)
Area_F = Area("F", etd_layout.shape[6].seq[8].x, etd_layout.shape[6].seq[8].y, angle=0.05235987755982988, offsetX=80, offsetY=8)


# add warehouse here
wh1 = House("WH", "Kho 1", etd_layout.shape[7],0,2,200,0.05235987755982988)
wh2 = House("WH", "kho 2", etd_layout.shape[8],2,0, 200, 0.05235987755982988)
wh3 = House("Kho", "Nha Kho", etd_layout.shape[9], 3,1,100,0.05235987755982988)
etd = Depot("etd", etd_layout, area=[Area_A, Area_B, Area_C, Area_E,Area_SC, Area_F], house=[wh1, wh2, wh3],container_scale=4.2)

shape: 99 60 100 123 39 138 1 169 1 528 13 966 22 966 24 992 321 1017 369 1015 365 823 523 825 496 568 549 327 551 1 119 11 117 60 99 60
shape: 450 124 451 205 70 205 70 137 100 137 99 125 450 124
shape: 451 263 451 388 70 388 70 263 451 263
shape: 499 496 384 499 382 443 498 439 499 496
shape: 279 457 282 497 65 509 62 468 279 457
shape: 316 678 323 838 61 850 53 691 316 678
shape: 325 898 35 911 38 991 126 988 127 1000 215 995 216 1007 272 1004 331 1001 325 898
shape: 329 623 58 636 52 522 324 509 329 623
shape: 368 511 382 810 496 805 485 505 368 511
shape: 336 6 186 10 188 57 338 51 336 6


In [7]:

jn = (depotEncoder().encode(etd))
ofile = open(outfile, "w")
ofile.write(jn)
ofile.close()

In [8]:
# doc json container
# file = open("data/cont3.json", "r")


In [9]:
# cont_array = json.load(file)
# for i in cont_array:
#     print(i)